In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
import json
import numpy as np
import itertools
from scripts.load_data import *

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor

import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [4]:
configuration = load_configuration("../configuration/configuration.json")

paths = configuration["paths"]
data_student_numbers_f = pd.read_excel(paths["path_student_count_first-years"])
data_student_numbers_h = pd.read_excel(paths["path_student_count_higher-years"])
data_student_numbers_v = pd.read_excel(paths["path_student_volume"])

In [5]:
combined_data_student_numbers = data_student_numbers_f.merge(data_student_numbers_h, on=["Croho groepeernaam", "Herkomst", "Collegejaar"])
combined_data_student_numbers = combined_data_student_numbers.rename(columns={"Aantal_studenten_x": "Aantal_studenten_f", "Aantal_studenten_y": "Aantal_studenten_h"})
combined_data_student_numbers

,Collegejaar,Croho groepeernaam,Herkomst,Aantal_studenten_f,Aantal_studenten_h
0,2012,M Geneeskunde,NL,180,868
1,2013,M Geneeskunde,NL,173,892
2,2014,M Geneeskunde,NL,175,916
3,2015,M Geneeskunde,NL,21,1086
4,2016,M Geneeskunde,NL,148,984
...,...,...,...,...,...
2156,2016,M Leraar Voorbereidend Hoger Onderwijs in Gods...,NL,1,1
2157,2014,M Leraar Voorbereidend Hoger Onderwijs in Natu...,NL,3,1
2158,2015,M Leraar Voorbereidend Hoger Onderwijs in Natu...,NL,1,1
2159,2017,M Leraar Voorbereidend Hoger Onderwijs in Natu...,NL,2,1


In [6]:
combined_data_student_numbers[combined_data_student_numbers["Croho groepeernaam"] == "B Sociologie"]

,Collegejaar,Croho groepeernaam,Herkomst,Aantal_studenten_f,Aantal_studenten_h
1819,2012,B Sociologie,NL,49,76
1820,2013,B Sociologie,NL,51,88
1821,2014,B Sociologie,NL,38,108
1822,2015,B Sociologie,NL,32,105
1823,2016,B Sociologie,NL,44,95
1824,2017,B Sociologie,NL,31,98
1825,2018,B Sociologie,NL,43,89
1826,2019,B Sociologie,NL,48,99
1827,2020,B Sociologie,NL,63,99
1828,2021,B Sociologie,NL,86,111


In [7]:
predict_year = 2023

look_back = 6
nr_of_epochs = 35

In [8]:
all_unique_years = np.sort(combined_data_student_numbers["Collegejaar"].unique())

In [9]:
total_MAE = 0.0
total_MAPE = 0.0
count = 0.0

for programme, origin in itertools.product(np.sort(combined_data_student_numbers["Croho groepeernaam"].unique()), np.sort(combined_data_student_numbers["Herkomst"].unique())):
    print(f"Start of: {programme}, {origin}")
    data = combined_data_student_numbers
    data = data.sort_values(by=["Collegejaar"])
    train = data[(data["Collegejaar"] < predict_year) & (data['Croho groepeernaam'] == programme) & (data["Herkomst"] == origin)]
    true_label = data[(data["Collegejaar"] == predict_year) & (data['Croho groepeernaam'] == programme) & (data["Herkomst"] == origin)]
    if len(true_label) == 0 or len(train) == 0:
        continue

    train_dataset = []
    for year in all_unique_years:
        year_train = train[train["Collegejaar"] == year]
        if len(year_train) == 0:
            train_dataset.append([0])
        else:
            train_dataset.append([year_train["Aantal_studenten_h"].iloc[0]])

    def create_dataset(dataset, look_back=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return np.array(dataX), np.array(dataY)

    scaler = MinMaxScaler(feature_range=(0, 1))
    train_dataset = scaler.fit_transform(train_dataset)

    X_train, Y_train = create_dataset(train_dataset, look_back)

    trainX = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))

    model = Sequential()
    model.add(LSTM(input_shape=(1, look_back), units=4))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, Y_train, epochs=nr_of_epochs, batch_size=1, verbose=0)

    test = train_dataset[-look_back:]

    reshaped_test = np.reshape(test, (1, 1, look_back))

    prediction = model.predict(reshaped_test, verbose=0)
    prediction = scaler.inverse_transform(prediction)
    
    mae = abs(prediction - true_label["Aantal_studenten_h"].iloc[0])
    mape = abs((prediction - true_label["Aantal_studenten_h"].iloc[0]) / true_label["Aantal_studenten_h"].iloc[0])
    print(f"{programme}, {origin}: {prediction} => {mae}")

    total_MAE += mae
    total_MAPE += mape
    count += 1.0

print(f"Final MAE: {total_MAE / count}")
print(f"Final MAPE: {total_MAPE / count}")

Start of: B Algemene Cultuurwetenschappen, EER


c:\Users\jjble\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


B Algemene Cultuurwetenschappen, EER: [[20.646328]] => [[15.353672]]
Start of: B Algemene Cultuurwetenschappen, NL


c:\Users\jjble\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


B Algemene Cultuurwetenschappen, NL: [[76.82928]] => [[0.17072296]]
Start of: B Algemene Cultuurwetenschappen, Niet-EER


...

M Tandheelkunde, NL: [[143.46947]] => [[12.530533]]
Start of: M Tandheelkunde, Niet-EER
Start of: M Theologie, EER
Start of: M Theologie, NL


c:\Users\jjble\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


M Theologie, NL: [[2.7006364]] => [[2.2993636]]
Start of: M Theologie, Niet-EER
Start of: M Theologie & Religiewetenschappen, EER
Start of: M Theologie & Religiewetenschappen, NL


c:\Users\jjble\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


M Theologie & Religiewetenschappen, NL: [[111.44352]] => [[29.55648]]
Start of: M Theologie & Religiewetenschappen, Niet-EER
Final MAE: [[27.242483]]
Final MAPE: [[0.3836243]]
